# Data Cleaning Pipeline

In [3]:
# Import Libraries

import pandas as pd
from pathlib import Path

## Load Data

In [16]:
pump_data_val = pd.read_csv("data/training_set_values.csv")
pump_data_lab = pd.read_csv("data/training_set_labels.csv")

print("Pump Data Values Shape: \n")
print(pump_data_val.shape)

print("\nPump Data Labels Shape: \n")
print(pump_data_lab.shape)

Pump Data Values Shape: 

(59400, 40)

Pump Data Labels Shape: 

(59400, 2)


In [12]:
pump_data_val.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [14]:
pump_data_lab.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [17]:
print("Check class Imbalance: ")
print(pump_data_lab.status_group.value_counts())

Check class balance: 
status_group
functional                 32259
non functional             22824
functional needs repair     4317
Name: count, dtype: int64


## Data Cleaning